In [11]:
import pandas as pd
import torch
import numpy as np

data = pd.read_csv('distillation-tower.csv')

X = data.drop('VapourPressure', axis=1)
X = X.iloc[:, 1:]
y = data['VapourPressure']

In [12]:
invt = data['InvTemp3']
vp = data['VapourPressure']

In [14]:
vp = np.log(vp)

In [17]:
from sklearn.linear_model import LinearRegression

invt = np.array(invt).reshape(-1, 1)
vp = np.array(vp)

# Initialize the linear regression model
model = LinearRegression()

# Fit the model
model.fit(invt, vp)

# Now you can use the model to make predictions
vp_pred = model.predict(invt)
X['Antoine'] = vp_pred

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
from torch.utils.data import TensorDataset, DataLoader

X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [20]:
import torch.nn as nn

class VaporPressureNet(nn.Module):
    def __init__(self):
        super(VaporPressureNet, self).__init__()
        self.fc1 = nn.Linear(X.shape[1], 64) 
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 1) 

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [21]:
model = VaporPressureNet()

criterion = nn.MSELoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 1000

for epoch in range(epochs):
    model.train() 
    running_loss = 0.0
    for inputs, targets in train_dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_dataloader.dataset)
    print(f'Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}')

model.eval() 
with torch.no_grad(): 
    predictions = []
    targets = []
    for inputs, true_values in test_dataloader:
        outputs = model(inputs)
        predictions.extend(outputs.squeeze().tolist())
        targets.extend(true_values.tolist())

from sklearn.metrics import r2_score

r2 = r2_score(targets, predictions)
print(f'R^2 Score on test set: {r2:.4f}')

Epoch 1/1000, Loss: 268.8005
Epoch 2/1000, Loss: 137.6983
Epoch 3/1000, Loss: 104.8832
Epoch 4/1000, Loss: 92.9264
Epoch 5/1000, Loss: 87.5352
Epoch 6/1000, Loss: 96.8755
Epoch 7/1000, Loss: 75.3903
Epoch 8/1000, Loss: 74.5744
Epoch 9/1000, Loss: 72.0940
Epoch 10/1000, Loss: 70.8816
Epoch 11/1000, Loss: 72.0188
Epoch 12/1000, Loss: 70.3453
Epoch 13/1000, Loss: 69.0743
Epoch 14/1000, Loss: 69.5025
Epoch 15/1000, Loss: 71.1042
Epoch 16/1000, Loss: 69.9876
Epoch 17/1000, Loss: 67.4722
Epoch 18/1000, Loss: 68.8808
Epoch 19/1000, Loss: 68.2383
Epoch 20/1000, Loss: 65.9489
Epoch 21/1000, Loss: 67.1018
Epoch 22/1000, Loss: 66.2404
Epoch 23/1000, Loss: 66.3503
Epoch 24/1000, Loss: 64.2316
Epoch 25/1000, Loss: 73.2836
Epoch 26/1000, Loss: 67.6576
Epoch 27/1000, Loss: 67.5767
Epoch 28/1000, Loss: 66.5281
Epoch 29/1000, Loss: 64.5664
Epoch 30/1000, Loss: 64.6431
Epoch 31/1000, Loss: 63.3186
Epoch 32/1000, Loss: 62.0669
Epoch 33/1000, Loss: 63.9344
Epoch 34/1000, Loss: 61.6968
Epoch 35/1000, Loss:

In [24]:
def evaluate(n):
    single_observation = X.iloc[n]
    single_observation = single_observation.apply(pd.to_numeric, errors='coerce').fillna(0)
    single_observation_tensor = torch.tensor(single_observation.values, dtype=torch.float32).unsqueeze(0)
    model.eval()
    with torch.no_grad():
        prediction = model(single_observation_tensor)        
    print("Predicted Vapour Pressure:", prediction.item())
    print(y[n])

In [28]:
evaluate(20)

Predicted Vapour Pressure: 39.90481185913086
40.2702
